# Notebook to test new functions

In [1]:
nest_dir = "neural_encoding_simulation_toolkit" 
test_images = "tutorial_images"

---

# Prepare all images

In [2]:
from tqdm import tqdm
from torchvision import transforms as trn
import numpy as np
import os
from PIL import Image

images_dir = os.path.join(test_images)
images_list = os.listdir(images_dir)
images_list.sort()
    
images = []
for img in tqdm(images_list):
    img_dir = os.path.join(images_dir, img)
    img = Image.open(img_dir).convert('RGB')
    # Center crop the images to square format, and resize them
    transform = trn.Compose([
        trn.CenterCrop(min(img.size)),
        trn.Resize((227,227))
    ])
    img = transform(img)
    img = np.asarray(img)
    img = img.transpose(2,0,1)
    images.append(img)
images = np.asarray(images)

# Print the images dimensions
print('\n\nImages shape:')
print(images.shape)
print('(Batch size × 3 RGB Channels x Width x Height)')

100%|██████████| 100/100 [00:00<00:00, 322.26it/s]



Images shape:
(100, 3, 227, 227)
(Batch size × 3 RGB Channels x Width x Height)


---

# How to run the code in general:

In [3]:
from nest import NEST

nest = NEST(nest_dir=nest_dir)

# List all available models and their versions
available_models = nest.list_models()
print(f"Available models: {available_models}")

# See what modalities are available
catalog = nest.get_model_catalog(print_format=True)
print(f"Model Catalog as Dict:: {catalog}")

Available models: ['fmri-nsd-fwrf', 'eeg-things_eeg_2-vit_b_32']
Available Modalities and Datasets:
• EEG
  └─ things_eeg_2

• FMRI
  └─ nsd

Model Catalog as Dict:: {'fmri': ['nsd'], 'eeg': ['things_eeg_2']}


In [4]:
# Load Encoding Model
fwrf_model = nest.get_encoding_model("fmri-nsd-fwrf", 
                                     subject=1, 
                                     device="cpu",
                                     selection={"roi": "V1"})


eeg_model = nest.get_encoding_model("eeg-things_eeg_2-vit_b_32", 
                                    subject=1,
                                    device="auto")


fwrf_silico = nest.encode(fwrf_model, images)
eeg_silico, eeg_metadata = nest.encode(eeg_model, images,return_metadata=True)

Model loaded on cpu for subject 1, ROI V1
Model loaded on cpu for subject 1


Encoding EEG responses: 100%|██████████| 1/1 [00:01<00:00,  1.73s/it, Encoded images=100, Total images=100]


---

# fwRF Model in more detail

In [5]:
from nest import NEST
nest = NEST(nest_dir=nest_dir)

# Choose a model of interest
model_id = "fmri-nsd-fwrf"  # Example model ID

# Get comprehensive model information
model_info = nest.describe(model_id)


🧠 Model: fmri-nsd-fwrf

Modality: fmri
Training dataset: NSD
Model architecture: feature-weighted receptive fields (fwrf)
Creator: Alessandro Gifford

📋 Description:
This model generates in silico fMRI responses to visual stimuli using feature-
weighted receptive fields (fwrf). It was trained on the Natural Scenes Dataset
(NSD), a large-scale 7T fMRI dataset of subjects viewing natural images. The
model extracts visual features using a convolutional neural network and maps
these features to brain activity  patterns across multiple visual regions of
interest (ROIs).  The model takes as input a batch of RGB images in the shape
[batch_size, 3, height, width], with pixel values ranging from 0 to 255 and
square dimensions (e.g., 224×224).

📥 Input:
  Type: numpy.ndarray
  Shape: ['batch_size', 3, 'height', 'width']
  Description: The input should be a batch of RGB images.
  Constraints:
    • Image values should be integers in range [0, 255]
    • Image dimensions (height, width) should be 

In [6]:
# Load Encoding Model
fwrf_model = nest.get_encoding_model("fmri-nsd-fwrf", 
                                     subject=1, 
                                     selection={"roi": "V1"})

Model loaded on cpu for subject 1, ROI V1


In [7]:
# Encode images
nest.encode(fwrf_model, images)

Encoding fMRI responses: 100%|██████████| 1/1 [00:00<00:00,  1.30it/s, Encoded images=100, Total images=100]


array([[-0.055599  ,  0.1633554 ,  0.0943293 , ...,  0.01447097,
         0.15747474,  0.01369774],
       [ 0.01819719,  0.07435445,  0.04977052, ..., -0.01315978,
         0.10419706,  0.03225161],
       [ 0.10229094,  0.21433882,  0.13944887, ...,  0.01901415,
         0.14813517,  0.00760403],
       ...,
       [ 0.04800354, -0.14399454, -0.28155515, ..., -0.02397447,
        -0.08016404,  0.00045429],
       [ 0.04686468,  0.07014671,  0.00538955, ..., -0.02637868,
         0.10473584,  0.02725478],
       [ 0.11645083,  0.02392432, -0.04246235, ..., -0.07162039,
        -0.0095483 , -0.01467158]], dtype=float32)

In [8]:
# Encode images and return metadata
nest.encode(fwrf_model, images, return_metadata=True)

Encoding fMRI responses: 100%|██████████| 1/1 [00:00<00:00,  1.29it/s, Encoded images=100, Total images=100]


(array([[-0.055599  ,  0.1633554 ,  0.0943293 , ...,  0.01447097,
          0.15747474,  0.01369774],
        [ 0.01819719,  0.07435445,  0.04977052, ..., -0.01315978,
          0.10419706,  0.03225161],
        [ 0.10229094,  0.21433882,  0.13944887, ...,  0.01901415,
          0.14813517,  0.00760403],
        ...,
        [ 0.04800354, -0.14399454, -0.28155515, ..., -0.02397447,
         -0.08016404,  0.00045429],
        [ 0.04686468,  0.07014671,  0.00538955, ..., -0.02637868,
          0.10473584,  0.02725478],
        [ 0.11645083,  0.02392432, -0.04246235, ..., -0.07162039,
         -0.0095483 , -0.01467158]], dtype=float32),
 {'fmri': {'ncsnr': array([0.10146447, 0.30089897, 0.2398103 , ..., 0.04201289, 0.22930188,
          0.06711598]),
   'roi_mask_volume': array([[[False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           [False, False, False, ..., False, False, False],
           ...,
           [False, False, F

In [9]:
# Just return metadata of model
fwrf_model.get_metadata()

{'fmri': {'ncsnr': array([0.10146447, 0.30089897, 0.2398103 , ..., 0.04201289, 0.22930188,
         0.06711598]),
  'roi_mask_volume': array([[[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          ...,
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False]],
  
         [[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          ...,
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False]],
  
         [[False, False, False, ..., False, False, False],
          [False, False, False, ..., False, False, False],
   

--- 

# Testing EEG Model

In [10]:
from nest import NEST
nest = NEST(nest_dir=nest_dir)


eeg_model = nest.get_encoding_model("eeg-things_eeg_2-vit_b_32", 
                                    subject=1)

Model loaded on cpu for subject 1


In [ ]:
nest.encode(eeg_model, images,return_metadata=True)

With Selection Parameter

In [11]:
feeg_model = nest.get_encoding_model("eeg-things_eeg_2-vit_b_32", 
                                    subject=1,
                                    device="auto",
                                    selection={"channels": ['F7']})

eeg_silico = nest.encode(eeg_model, images)

print(eeg_silico.shape)

Model loaded on cpu for subject 1


Encoding EEG responses: 100%|██████████| 1/1 [00:01<00:00,  1.79s/it, Encoded images=100, Total images=100]

(100, 4, 63, 140)


In [12]:
vector = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
          0, 0, 0, 0, 0, 0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 ,0]


eeg_model = nest.get_encoding_model("eeg-things_eeg_2-vit_b_32", 
                                    subject=1,
                                    device="auto",
                                    selection={"channels": ['F7'],
                                               "timepoints": vector})

eeg_silico = nest.encode(eeg_model, images)

print(eeg_silico.shape)

Model loaded on cpu for subject 1


Encoding EEG responses: 100%|██████████| 1/1 [00:01<00:00,  1.77s/it, Encoded images=100, Total images=100]

(100, 4, 1, 50)


---

### You can describe a model in two ways

In [14]:
description = nest.describe("eeg-things_eeg_2-vit_b_32")

🧠 Model: eeg-things_eeg_2-vit_b_32

Modality: eeg
Training dataset: things_eeg_2
Model architecture: vision transformer (ViT-B/32)
Creator: Alessandro Gifford

📋 Description:
This model generates in silico EEG responses to visual stimuli using a vision
transformer model. It was trained on the THINGS-EEG-2 dataset, which contains
EEG recordings from subjects viewing images of everyday objects. The model
extracts visual features using a pre-trained ViT-B/32 transformer, applies
dimensionality reduction, and then predicts EEG responses across all channels
and time points.  The model takes as input a batch of RGB images in the shape
[batch_size, 3, height, width], with pixel values ranging from 0 to 255 and
square dimensions (e.g., 224×224).

📥 Input:
  Type: numpy.ndarray
  Shape: ['batch_size', 3, 'height', 'width']
  Description: The input should be a batch of RGB images.
  Constraints:
    • Image values should be integers in range [0, 255]
    • Image dimensions (height, width) should

In [16]:
description = eeg_model.describe()

🧠 Model: eeg-things_eeg_2-vit_b_32

Modality: eeg
Training dataset: things_eeg_2
Model architecture: vision transformer (ViT-B/32)
Creator: Alessandro Gifford

📋 Description:
This model generates in silico EEG responses to visual stimuli using a vision
transformer model. It was trained on the THINGS-EEG-2 dataset, which contains
EEG recordings from subjects viewing images of everyday objects. The model
extracts visual features using a pre-trained ViT-B/32 transformer, applies
dimensionality reduction, and then predicts EEG responses across all channels
and time points.  The model takes as input a batch of RGB images in the shape
[batch_size, 3, height, width], with pixel values ranging from 0 to 255 and
square dimensions (e.g., 224×224).

📥 Input:
  Type: numpy.ndarray
  Shape: ['batch_size', 3, 'height', 'width']
  Description: The input should be a batch of RGB images.
  Constraints:
    • Image values should be integers in range [0, 255]
    • Image dimensions (height, width) should